In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import random
from shutil import copyfile

# Loading the Inception V3 model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pretrained_model = InceptionV3(input_shape=(150,150,3),include_top=False,weights=local_weights_file)

for layer in pretrained_model.layers:
    layer.trainable = False

#pretrained_model.load_weights(local_weights_file)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
last_layer.output_shape

In [ ]:
# Callbacks for stopping training after a certain % of accuracy is reached
class MyCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_stop(self,epoch,logs={}):
        if (logs.get['acc'] > .95):
            
            print ("Stopping the training as 95% acccuracy is reached")
            self.model.stop_training= False

In [ ]:
from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dropout(rate=0.2)(x)
x = layers.Dense(1,activation='sigmoid')(x)

# Connect the initial layers and the changed layers
model = Model(pretrained_model.input,x)

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
def split_data(Source,Training,Testing,Split_size):
    files = []
    for filename in os.listdir(Source):
        file = Source + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print (filename + " " + "is zero size file")
    # Splitting based on random set
    training_length = int(len(files)*Split_size)
    testing_length = int(len(files)-training_length)
    shuffled_set = random.sample(files,len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]
    
    #copying the file 
    for filename in training_set:
        this_file = Source + filename
        destination = Training + filename
        copyfile(this_file,destination)
    for filename in testing_set:
        this_file = Source + filename
        destination = Testing + filename
        copyfile(this_file,destination)
        
Cat_Source_Dir = 'Data/PetImages/Cat/'
Cat_Training_Dir = 'Data/cats-v-dogs/training/cats/'
Cat_Testing_Dir = 'Data/cats-v-dogs/testing/cats/'
Dog_Source_Dir = 'Data/PetImages/Dog/'
Dog_Training_Dir = 'Data/cats-v-dogs/training/dogs/'
Dog_Testing_Dir = 'Data/cats-v-dogs/testing/dogs/'

split_size = 0.9
split_data(Cat_Source_Dir,Cat_Training_Dir,Cat_Testing_Dir,split_size)
split_data(Dog_Source_Dir,Dog_Training_Dir,Dog_Testing_Dir,split_size)

In [ ]:
print (len(os.listdir('Data/cats-v-dogs/training/dogs/')))
print (len(os.listdir('Data/cats-v-dogs/training/cats/')))
print (len(os.listdir('Data/cats-v-dogs/testing/dogs/')))
print (len(os.listdir('Data/cats-v-dogs/testing/cats/')))

In [ ]:
callbacks = MyCallbacks()

# Data Augmentation of Dogs and Cats data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
Training_Dir = 'Data/cats-v-dogs/training'

train_gen = ImageDataGenerator(rescale=1./255.,
                              rotation_range=50,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               vertical_flip=True,
                               horizontal_flip= True)

training_generator = train_gen.flow_from_directory(directory=Training_Dir,target_size=(150,150),class_mode='binary',batch_size=32)

Validation_Dir ='Data/cats-v-dogs/testing/'
validation_gen = ImageDataGenerator(rescale=1.0/255.,
                                    rotation_range=50,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   vertical_flip=True,
                                   horizontal_flip= True
                                   )
validation_generator = validation_gen.flow_from_directory(directory=Validation_Dir,batch_size=32,class_mode='binary',target_size=(150,150))

In [ ]:
history = model.fit_generator(training_generator,epochs=2,verbose=1,callbacks=[callbacks],validation_data=validation_generator)